In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.shape

(404290, 6)

In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
new_df = df.sample(50000)

In [6]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [7]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
228596,Why do dogs like to tear the innards out of st...,When were the first stuffed animals made?
183567,Which type of nose I have?,What type of nose do I have?
18714,What is the best way to solve a problem?,What is the best way to solve your problems?
366160,Why do I dream every night?,Why don't I dream at night?
108012,Should I trust Apple or Google with my informa...,Should I trust Apple with storing all of my da...


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features = 5000)
q1_arr,q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [10]:
temp_df1 = pd.DataFrame(q1_arr, index = ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index = ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis = 1)
temp_df.shape

(50000, 10000)

In [11]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [12]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4991,4992,4993,4994,4995,4996,4997,4998,4999,is_duplicate
228596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
183567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18714,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
366160,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
108012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    temp_df.iloc[:, 0:-1].values,
    temp_df.iloc[:, -1].values,
    test_size=0.2,
    random_state=42
)


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7596

In [17]:
!pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/150.0 MB 2.1 MB/s eta 0:01:12
   ---------------------------------------- 0.8/150.0 MB 2.6 MB/s eta 0:00:58
   ---------------------------------------- 1.0/150.0 MB 1.3 MB/s eta 0:01:56
   ---------------------------------------- 1.0/150.0 MB 1.3 MB/s eta 0:01:56
   ---------------------------------------- 1.6/150.0 MB 1.3 MB/s eta 0:01:56
   ---------------------------------------- 1.8/150.0 MB 1.3 MB/s eta 0:01:51
    --------------------------------------- 2.1/150.0 MB 1.3 MB/s eta 0:01:50
    --------------------------------------- 2.6/150.0 MB 1.4 MB/s eta 0:01:43
    --------------------------------------- 2.9/150.0 MB 1.5 MB/s eta 0:01:40
    --------------------------------------- 3.4/150.0 MB 1.5 MB/s eta 0:01:35
    --------------------------------------- 3.7/150.0 MB 1.6 MB/s eta 0:01:34


In [21]:
from xgboost import XGBClassifier
xgb =  XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test, y_pred)

0.7389